In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random


In [124]:
def init_shepherd(L):
    # find initial position of shepheard at the lower left quater of LxL field
    x = random.sample(range(0,math.floor(L/2)),1)
    y = random.sample(range(0,math.floor(L/2)),1)
    return np.array([x,y])

def init_sheep(N, L):
    # find initial positions of N sheeps at top right quater of LxL field
    X = random.sample(range(math.floor(L/2),L),N)
    Y = random.sample(range(math.floor(L/2),L),N)
    l = []
    for i in range(N):
        l.append(np.array([X[i],Y[i]]))
    
    return np.asarray(l)

def sheep_sheep_distance(N, pos):
    # pos : Nx2 array
    # return NxN numpy array
    dist = np.zeros((N,N))
    
    for i in range(N):
        for j in range(N):
            dist[i,j] = np.linalg.norm(pos[i]-pos[j])
    return dist 
        

def shep_sheep_distance(N, pos, shep):
    # return size N numpy array
    dist = np.zeros(N)
    
    for i in range(N):
        dist[i] = np.linalg.norm(shep - pos[i])
    
    return dist

def calculate_n_nearest(i, n, distances, positions):
    indices = distances[i].argsort()[(-1*n):][::-1]
    locs = [positions[i] for i in indices]
    return np.asarray(locs)

def calculate_LCM(neighbours):
    n = len(neighbours)
    vec = np.sum(neighbours, axis = 0)
    return vec/n

def condition_fn(gcm, fn, sheeps, N):
    for i in range(N):
        d = np.linalg.norm(gcm-sheeps[i])
        if d >= fn:
            return False
    return True
    
#one = init_sheep(10,100)
#two = init_shepherd(100)

#ss = sheep_sheep_distance(10, one)
#calculate_n_nearest(1,3,ss, one)
#print(shep_sheep_distance(N,one,two))

In [217]:
def main(L, N, n, r_s, r_a, rho_a, c, rho_s, h, e, delta, p, delta_s, pd_distance, pc_distance, e_s, time):
    # L : size of field
    # N : no. of sheep
    # r_s : shep detection distance
    # r_a : agent to agent interaction distance
    # rho_a : relative inter-sheep repulsion strength
    # c : relative attaction strength to n nearest neighbours
    # rho_s : strength of repultion from shep
    # h : inertial strength
    # e : noise strength
    # delta : sheep displacement per timestep
    # p : probability of moving while grazing
    # delta_s : shep displacement per timestep
    # pd_distance : for shep driving
    # pc_distance : for shep centering
    # e_s : noise for shep
    # time: no. of timesteps
    
    positions_sheep = {}
    positions_sheep[0] = init_sheep(N, L) # numpy array of size Nx2
        
    positions_shep = [] # list of Numpy array of size 2
    positions_shep.append(init_shepherd(L)) 
    
    for k in range(1,time+1):
        
        #print(positions_shep[k-1])
        aa_dist = sheep_sheep_distance(N, positions_sheep[k-1])
        as_dist = shep_sheep_distance(N, positions_sheep[k-1], positions_shep[k-1])
        
        positions_sheep[k] = np.zeros((N,2))
        
        # decide new location of every sheep
        
        # check distance of sheep from every other sheep
        for i in range(N): 
    
            Ra = np.array([0.0,0.0])
            for j in range(N):
                
                if j != i and aa_dist[i][j] <= r_a:
                    
                    Ra += (positions_sheep[k-1][i]-positions_sheep[k-1][j])/np.linalg.norm(positions_sheep[k-1][i]-positions_sheep[k-1][j])
            
            if np.linalg.norm(Ra) == 0:
                Ra_uv = Ra
            else:
                Ra_uv = Ra/np.linalg.norm(Ra) # unit vector from inter sheep repultion
            #print(Ra_uv)
            # calculate direction of resultant repultion from the neighbouring sheep- rho_a*(resultant direction)
            H = rho_a * Ra_uv
           
            # check the distance of sheep i from shepherd
            if as_dist[i] >= r_s:
            
            # if not under influence
                toss = random.random()
                if toss <= p:
                    x = random.uniform(0,L)
                    y = random.uniform(0,L)
                    direc = np.array([x,y])
                    if (np.linalg.norm(direc)) == 0:
                        print(2)
                    uvec = direc/(np.linalg.norm(direc))
                    H += uvec
            else:
                
                neighbours = calculate_n_nearest(i, n, aa_dist, positions_sheep[k-1]) # list of coordinates of n nearest neighbours
                
                LCM = calculate_LCM(neighbours)
                if np.linalg.norm(LCM - positions_sheep[k-1][i]) == 0:
                    print(3)
                C = (LCM - positions_sheep[k-1][i])/np.linalg.norm(LCM - positions_sheep[k-1][i])
                
                H += c * C
                
                # attracted to LCM of n nearest neighbours - c*(resultant direction)
                if np.linalg.norm(positions_sheep[k-1][i].reshape(2,1) - positions_shep[k-1].reshape(2,1)) == 0:
                    print(4)
                
                Rs = (positions_sheep[k-1][i].reshape(2,1) - positions_shep[k-1].reshape(2,1))/np.linalg.norm(positions_sheep[k-1][i].reshape(2,1) - positions_shep[k-1].reshape(2,1))
                
                H += (rho_s * np.ravel(Rs))
                
                # repulsion from shepherd - rho_s*(resultant direction)
            epsilon = np.random.rand(2)
            if np.linalg.norm(epsilon) == 0:
                print(5)
            epsilon_uv = epsilon/np.linalg.norm(epsilon)
            #print(positions_sheep[k-1][i] + e*epsilon_uv,type(H[0]))
            H += h* positions_sheep[k-1][i] + e*epsilon_uv
            
            if np.linalg.norm(H) == 0:
                print(6)
            H = H/np.linalg.norm(H)
            
            # new agent location = old agent location + delta*(H)
            
            positions_sheep[k][i] = positions_sheep[k-1][i] - delta*(H) 
        
        
        
        # decide new location of shepherd
        gcm = calculate_LCM(positions_sheep[k-1])
        
        fn = (r_a * (N ** (2/3)))
        speed = delta_s
        
        for i in as_dist:
            if i <= (3 * r_a):
                speed = 0
                break
        if speed != 0:
            
            if condition_fn(gcm, fn, positions_sheep[k-1], N):
            
                pd = gcm + ((gcm/np.linalg.norm(gcm)) * pd_distance)
            
                
            
                S = (pd.reshape(2,1) - positions_shep[k-1])/np.linalg.norm(pd.reshape(2,1) - positions_shep[k-1])*speed
                
                #shepherd goes towards pd
            
            else:
                ind_farthest = shep_sheep_distance(N, positions_sheep[k-1], gcm).argmax()
                af = positions_sheep[k-1][ind_farthest]
                pc = af + ((af-gcm)/np.linalg.norm(af-gcm) * pc_distance)
                
                S = (pc.reshape(2,1) - positions_shep[k-1])/np.linalg.norm(pc.reshape(2,1) - positions_shep[k-1])*speed
                
                #shepherd goes towards pc
        
            positions_shep.append(positions_shep[k-1] + S)
        else:
            positions_shep.append(positions_shep[k-1])
    
    return positions_sheep, positions_shep
    
    
    
    

In [224]:
L = 150
N = 20
n = 2
r_s = 65
r_a = 2
rho_a = 2
c = 1.05
rho_s = 1
h = 0.5
e = 0.3
delta = 1
p = 0.05
delta_s = 1.5
pd_distance = r_a * math.sqrt(N)
pc_distance = r_a
e_s = 0.3
time = 500
sheep , shepherd = main(L, N, n, r_s, r_a, rho_a, c, rho_s, h, e, delta, p, delta_s, pd_distance, pc_distance, e_s, time)

In [236]:
c = 0
sheeps = {}
for i in range(time):
    for j in range(N):
        sheeps[c] = [i, sheep[i][j][0],sheep[i][j][1]]
        c+=1
sheps = {}
for i in range(time):
    sheps[i] = [i, shepherd[i][0][0],shepherd[i][1][0]]
print(len(sheps.keys()))


500


In [237]:
import pandas as pd
df_sheep = pd.DataFrame.from_dict(sheeps, orient='index', columns=["t", 'x', 'y'])
df_shep =  pd.DataFrame.from_dict(sheps, orient='index', columns=["t", 'x', 'y'])


In [242]:

import plotly.graph_objects as go
frames = []
for t in range(time):
    trace_sheep = go.Scatter(
            x=df_sheep.query('t =='+str(t)).x,
            y=df_sheep.query('t =='+str(t)).y)
    trace_shep = go.Scatter(
            x=df_shep.query('t =='+str(t)).x,
            y=df_shep.query('t =='+str(t)).y)
    frame = go.Frame(data = [trace_sheep, trace_shep], layout={'title': "Time Step = "+str(t)})
    frames.append(frame)


In [244]:
fig = go.Figure(
    data=[go.Scatter(
                x=df_sheep.query('t =='+str(0)).x,
                y=df_sheep.query('t =='+str(0)).y,
                marker = dict(size = 10),
                marker_color = 'rgb(125, 0, 50)',
                name = "Sheep",
                opacity = 0.9, connectgaps=False, mode='markers'), 
          go.Scatter(
                x=df_shep.query('t =='+str(0)).x,
                y=df_shep.query('t =='+str(0)).y,
                marker = dict(size = 15),
                name = "Shepherd",
                marker_color = 'rgb(0, 0, 0)',
                opacity = 0.8, connectgaps=False, mode='markers')],
    layout=go.Layout(
        xaxis=dict(range=[-50, 150]),
        yaxis=dict(range=[-50, 150]),
        xaxis_title="X Coordinate",
        yaxis_title="Y Coordinate",
        title="MAS ROCKS!",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None])])]
    ),
    frames=frames
)

In [245]:
fig.show()

In [247]:
import plotly
plotly.offline.plot(fig, filename='sheep.html')

'sheep.html'